## Resource Allocation Example

Assume we have to assign resources of $m$ classes to $n$ kinds of jobs. This resource allocation is encoded in $X \in \mathbb{R}^{n \times m}$, with $X_{i,j}$ denoting the amount of resource $j$ allocated to job $i$. Given the utility matrix $W \in \mathbb{R}^{n \times m}$, we want to solve the optimization problem

\begin{equation}
\begin{array}{ll}
\text{maximize} \quad &\mathrm{tr} \left( W^T X \right)\\
\text{subject to} \quad &X^\mathrm{min} \leq X \leq X^\mathrm{max} \\
&X^T \mathbb{1} \leq r,
\end{array}
\end{equation}

with variable $X \in \mathbb{R}^{n \times m}$. The minimum and maximum amounts of resources to be allocated are denoted by $X^\mathrm{min} \geq 0$ and $X^\mathrm{max} \geq X^\mathrm{min}$, respectively, while $r$ is the vector of available resources. The problem is feasible if $\left(X^\mathrm{min}\right)^T \mathbb{1} \leq r$ and $X^\mathrm{min} \leq X^\mathrm{max}$.

Let's define the corresponding CVXPY problem.

In [1]:
import cvxpy as cp
import numpy as np

# define dimensions
n, m = 100, 10

# define variable
X = cp.Variable((n, m), name='X')

# define parameters
W = cp.Parameter((n, m), name='W')
X_min = cp.Parameter((n, m), name='X_min')
X_max = cp.Parameter((n, m), name='X_max')
r = cp.Parameter(m, name='r')

# define objective
objective = cp.Maximize(cp.trace(W.T@X))

# define constraints
constraints = [X_min <= X, X<= X_max, 
               X.T@np.ones(n) <= r]

# define problem
problem = cp.Problem(objective, constraints)

Assign parameter values and solve the problem.

In [2]:
np.random.seed(0)

W.value = np.ones((n, m)) + 0.1*np.random.rand(n, m)
X_min.value = np.random.rand(n, m)
X_max.value = 10 + np.random.rand(n, m)
r.value = np.matmul(X_min.value.T, np.ones(n)) + 10*np.random.rand(m)

val = problem.solve(solver='OSQP')

Generating C source for the problem is as easy as:

In [3]:
import sys
sys.path.append('../')
import cvxpygen as cpg

cpg.generate_code(problem, code_dir='resource_code')

Generating code ...
-----------------------------------------------------------------
           OSQP v0.6.2  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 1000, constraints m = 2010
          nnz(P) + nnz(A) = 3000
settings: linear system solver = qdldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

Getting workspace from OSQP object... 				[done]
Creating target directories... 					[done]
Copying OSQP sources... 					[done]
Generating customized code... 					[done]
Creating project...
-- The C compile

ld: warning: object file (/Users/maximilianschaller/Documents/Stanford/codegen/examples/resource_code/c/build/out/libcpg.a(cpg_workspace.c.o)) was built for newer macOS version (11.0) than being linked (10.9)
ld: warning: object file (/Users/maximilianschaller/Documents/Stanford/codegen/examples/resource_code/c/build/out/libcpg.a(cpg_solve.c.o)) was built for newer macOS version (11.0) than being linked (10.9)
ld: warning: object file (/Users/maximilianschaller/Documents/Stanford/codegen/examples/resource_code/c/build/out/libcpg.a(osqp.c.o)) was built for newer macOS version (11.0) than being linked (10.9)
ld: warning: object file (/Users/maximilianschaller/Documents/Stanford/codegen/examples/resource_code/c/build/out/libcpg.a(workspace.c.o)) was built for newer macOS version (11.0) than being linked (10.9)
ld: warning: object file (/Users/maximilianschaller/Documents/Stanford/codegen/examples/resource_code/c/build/out/libcpg.a(error.c.o)) was built for newer macOS version (11.0) than 

Now, you can use a python wrapper around the generated code as a custom CVXPY solve method.

In [6]:
from resource_code.cpg_solver import cpg_solve
import numpy as np
import pickle
import time

# load the serialized problem formulation
with open('resource_code/problem.pickle', 'rb') as f:
    prob = pickle.load(f)

# assign parameter values
np.random.seed(0)
prob.param_dict['W'].value = np.ones((n, m)) + 0.1*np.random.rand(n, m)
prob.param_dict['X_min'].value = np.random.rand(n, m)
prob.param_dict['X_max'].value = 10 + np.random.rand(n, m)
prob.param_dict['r'].value = np.matmul(prob.param_dict['X_min'].value.T, np.ones(n)) + 10*np.random.rand(m)

# solve problem conventionally
t0 = time.time()
# CVXPY chooses eps_abs=eps_rel=1e-5, max_iter=10000, polish=True by default,
# however, we choose the OSQP default values here, as they are used for code generation as well
val = prob.solve(solver='OSQP', eps_abs=1e-3, eps_rel=1e-3, max_iter=4000, polish=False)
t1 = time.time()
print('\nPython solve time:', 1000*(t1-t0), 'ms')
print('Python objective function value:', val)

# solve problem with C code via python wrapper
prob.register_solve('CPG', cpg_solve)
t0 = time.time()
val = prob.solve(method='CPG')
t1 = time.time()
print('\nC solve time:', 1000*(t1-t0), 'ms')
print('C objective function value:', val)


Python solve time: 268.4500217437744 ms
Python objective function value: 595.9131811632495

C solve time: 4.148006439208984 ms
C objective function value: 595.7180234574506
